In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("CSVUnion") \
    .getOrCreate()


day1 = spark.read.csv("C:/Users/Lenovo/Downloads/CPU-LOG-DATA/CpuLogData2019-09-16.csv", header=True, inferSchema=True)
day2 = spark.read.csv("C:/Users/Lenovo/Downloads/CPU-LOG-DATA/CpuLogData2019-09-17.csv", header=True, inferSchema=True)
day3 = spark.read.csv("C:/Users/Lenovo/Downloads/CPU-LOG-DATA/CpuLogData2019-09-18.csv", header=True, inferSchema=True)
day4 = spark.read.csv("C:/Users/Lenovo/Downloads/CPU-LOG-DATA/CpuLogData2019-09-19.csv", header=True, inferSchema=True)
day5 = spark.read.csv("C:/Users/Lenovo/Downloads/CPU-LOG-DATA/CpuLogData2019-09-20.csv", header=True, inferSchema=True)

combined_days = day1.union(day2).union(day3).union(day4).union(day5)
combined_days_sensor = combined_days.select("DateTime","user_name","keyboard","mouse")
combined_days.show()





+-------------------+---------+----------------+-------------+-----------+----------------+----------------------------------------+---------------------------------+-------------------------------+-----------------------+-----------------------+------------------------+-------------------+------------------+------------------+--------------------+----------------------+---------------------+--------------------+--------------------+----------------------+-----------------+----------------+----------------+---------------+----------------+---------------+----------------+----------------------------+--------------------------+----------------------------+--------------------+------------------------+----------------------+-------------------------+--------------------------------------+------------------------------------+---------------------------------------------------+---------------------------------------------------+--------------------+--------------------+--------+------+----

In [3]:
combined_days_sensor.show()

+-------------------+--------------------+--------+------+
|           DateTime|           user_name|keyboard| mouse|
+-------------------+--------------------+--------+------+
|2019-09-16 12:55:03|rahilstar11@gmail...|     0.0|   0.0|
|2019-09-16 12:55:02|salinabodale73@gm...|  2919.5| 888.0|
|2019-09-16 12:55:01|bhagyashrichalke2...|   144.0|2886.0|
|2019-09-16 13:00:01|bhagyashrichalke2...|    21.0|  44.0|
|2019-09-16 13:00:01|  iamnzm@outlook.com|    41.0|8251.0|
|2019-09-16 13:00:01|deepshukla292@gma...|   249.5|4266.0|
|2019-09-16 13:00:02|salinabodale73@gm...|   135.5| 692.0|
|2019-09-16 13:00:04|sharlawar77@gmail...|   303.0| 243.0|
|2019-09-16 13:00:03|rahilstar11@gmail...|    22.5| 170.0|
|2019-09-16 13:05:01|salinabodale73@gm...|   179.5| 108.0|
|2019-09-16 13:05:02|bhagyashrichalke2...|   387.0| 676.0|
|2019-09-16 13:05:01|deepshukla292@gma...|   165.0|5701.0|
|2019-09-16 13:05:02|  iamnzm@outlook.com|    52.0|7971.0|
|2019-09-16 13:05:04|sharlawar77@gmail...|    17.0|  72.

find lowest average hours

In [4]:
from pyspark.sql.functions import avg
average_hours_df = combined_days_sensor.groupBy("user_name").agg(avg("keyboard").alias("avg_keyboard"), avg("mouse").alias("avg_mouse"))
lowest_avg_hours_df = average_hours_df.orderBy("avg_keyboard", "avg_mouse")
lowest_avg_hours_df.show()

+--------------------+------------------+------------------+
|           user_name|      avg_keyboard|         avg_mouse|
+--------------------+------------------+------------------+
|markfernandes66@g...| 36.90700483091788| 851.4710144927536|
|damodharn21@gmail...|49.264150943396224| 2538.245283018868|
|rahilstar11@gmail...|50.554083885209714| 216.5121412803532|
|  iamnzm@outlook.com|53.262042389210016| 754.6782273603083|
|bhagyashrichalke2...| 84.03994845360825| 1645.971649484536|
|sharlawar77@gmail...| 92.08921161825727|  442.597510373444|
|deepshukla292@gma...|118.11333333333333|2278.9755555555557|
|salinabodale73@gm...| 763.3644957983194| 70.67647058823529|
+--------------------+------------------+------------------+



find highest average hours

In [5]:
average_hours_df = combined_days_sensor.groupBy("user_name").agg(avg("keyboard").alias("avg_keyboard"), avg("mouse").alias("avg_mouse"))
highest_avg_hours_df = average_hours_df.orderBy("avg_keyboard", "avg_mouse", ascending=False)
highest_avg_hours_df.show()


+--------------------+------------------+------------------+
|           user_name|      avg_keyboard|         avg_mouse|
+--------------------+------------------+------------------+
|salinabodale73@gm...| 763.3644957983194| 70.67647058823529|
|deepshukla292@gma...|118.11333333333333|2278.9755555555557|
|sharlawar77@gmail...| 92.08921161825727|  442.597510373444|
|bhagyashrichalke2...| 84.03994845360825| 1645.971649484536|
|  iamnzm@outlook.com|53.262042389210016| 754.6782273603083|
|rahilstar11@gmail...|50.554083885209714| 216.5121412803532|
|damodharn21@gmail...|49.264150943396224| 2538.245283018868|
|markfernandes66@g...| 36.90700483091788| 851.4710144927536|
+--------------------+------------------+------------------+



idel hours

In [7]:
combined_days_sensor.createOrReplaceTempView("user_activity")
sql_query = """
    SELECT user_name, COUNT(*) AS idle_hours
    FROM user_activity
    WHERE keyboard = 0 AND mouse = 0
    GROUP BY user_name
    ORDER BY idle_hours DESC
"""
result= spark.sql(sql_query)
result.show()

+--------------------+----------+
|           user_name|idle_hours|
+--------------------+----------+
|rahilstar11@gmail...|       129|
|  iamnzm@outlook.com|       127|
|salinabodale73@gm...|       118|
|markfernandes66@g...|       112|
|sharlawar77@gmail...|       109|
|deepshukla292@gma...|        78|
|bhagyashrichalke2...|        62|
|damodharn21@gmail...|        45|
+--------------------+----------+



late comers

In [8]:
spark.sql("""select user_name, late_comers, dense_rank() over (ORDER BY late_comers desc) as rank 
          from (select user_name, count(*) as late_comers from user_activity where DateTime > "2019-09-17 08:31:00" 
          group by user_name) as counts order by late_comers desc""").show()

+--------------------+-----------+----+
|           user_name|late_comers|rank|
+--------------------+-----------+----+
|  iamnzm@outlook.com|        443|   1|
|markfernandes66@g...|        414|   2|
|sharlawar77@gmail...|        404|   3|
|salinabodale73@gm...|        385|   4|
|rahilstar11@gmail...|        381|   5|
|deepshukla292@gma...|        372|   6|
|bhagyashrichalke2...|        309|   7|
|damodharn21@gmail...|        159|   8|
+--------------------+-----------+----+



uploading it to mssql server

In [9]:
from pyspark import SparkContext, SparkConf

In [10]:
from pyspark.sql import SparkSession

In [12]:
spark = SparkSession.builder\
.appName("Write to Sql server")\
.config("spark.driver.extraClassPath", "C:/sqljdbc_12.6.1.0_enu/sqljdbc_12.6/enu/mssql-jdbc-12.6.1.jre11.jar") \
.getOrCreate()


In [13]:

jdbc_url = "jdbc:sqlserver://your_server:your_port;databaseName=your_database"
connection_properties = {
    "user": "SQLEXPRESS",
    "password": "Palash@123",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}


table_name = "calltable"


jdbc_url.write.jdbc(url=jdbc_url, table=table_name, mode="overwrite", properties=connection_properties)



NameError: name 'df' is not defined